### Deploy Dev Machine Quantizer on RunPOD

<a href="https://colab.research.google.com/github/kyledinh/gpt-prive/blob/main/notebookds/runpod/deploy-dev-machine-quantizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install and import dependencies

In [1]:
%pip install -Uqqq pip --progress-bar off
%pip install -qqq runpod==0.10.0 --progress-bar off
%pip install -qqq text-generation==0.6.0 --progress-bar off
%pip install -qqq requests==2.31.0 --progress-bar off

import requests
import runpod
from text_generation import Client

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Setup .env variables
- Create an `.env` with your specific token from your account.
- https://www.runpod.io/console/user/settings
- https://huggingface.co/settings/tokens

```
RUNPOD_API_KEY=9M5OM37OK3N5OM37OK3N5OM37OK3NJ875OM37
HF_ACCESS_TOKEN=hf_ENv5OM37OK3N5OM37OK3N5OM37OK3N5OM37OK3NT
```

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

HF_ACCESS_TOKEN = os.getenv("HF_ACCESS_TOKEN", "hf-some-token")
RUNPOD_API_KEY = os.getenv("RUNPOD_API_KEY", "add-here-if-not-set-in-env-file")
 
assert HF_ACCESS_TOKEN.startswith("hf_"), "This doesn't look like a valid Hugging Face Token"
assert not RUNPOD_API_KEY.startswith("add-here"), "This doesn't look like a valid Runpod API Key"

runpod.api_key = RUNPOD_API_KEY 
print("HF_ACCESS_TOKEN: " + HF_ACCESS_TOKEN[0:6])
print("RUNPOD_API_KEY: " + runpod.api_key[0:6])

HF_ACCESS_TOKEN: hf_ENv
RUNPOD_API_KEY: 9MJ87C


#### Deploy to RunPod
- https://www.runpod.io/console/pods

In [3]:
## TODO: update with proper settings.....

podname = "dev-machine-quantizer-48VRAM"
envs = {"HUGGING_FACE_HUB_TOKEN":HF_ACCESS_TOKEN, "QUANTIZE":"gptq"}

model = "TheBloke/Llama-2-7B-GPTQ"
gpu_type_id = "NVIDIA RTX A6000" # 48GB VRAM $0.79/hr
gpu_count = 1

# data_center_id="EU-RO-1" | "EU-CZ-1" | "US-KS-1" | "US-KS-2",

pod = runpod.create_pod(
    name=podname,
    image_name="runpod/pytorch:2.0.1-py3.10-cuda11.8.0-devel",
    gpu_type_id=gpu_type_id,
    cloud_type="SECURE",
    data_center_id="US-KS-1",
    gpu_count=gpu_count,
    volume_in_gb=50,
    container_disk_in_gb=10,
    ports="80/http,29500/http",
    volume_mount_path="/data",
    env=envs,
)


In [ ]:
SERVER_URL = f'https://{pod["id"]}-80.proxy.runpod.net'
print(SERVER_URL)

In [ ]:
# ssh into shell of instance 
# apt update 
# apt-get -y install vim

## API

In [ ]:
runpod.terminate_pod(pod["id"])

## References

- https://www.runpod.io/console/gpu-secure-cloud
- https://docs.runpod.io/docs/get-gpu-types
- https://github.com/facebookresearch/llama
- https://github.com/huggingface/text-generation-inference
- https://github.com/runpod/runpod-python
